In [1]:
import numpy as np
import pandas as pd
import re, json, csv, requests, time, glob, tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from collections import Counter

In [2]:
def scrape_one_new(html, url):
    soup = BeautifulSoup(html, "html.parser")
    json_data = json.loads(soup.find_all("script", type="application/ld+json")[-1].text)
    title = json_data.get('headline', soup.find('span', class_='contentTitle').text)
    date = json_data.get('datePublished', re.search(r'datetime:.*?(\d{4}-\d{2}-\d{2}T\d{2}:\d{2})', str(html)).group(1))
    date_m = json_data.get('dateModified', '')
    genre = json_data.get('genre', [])
    if genre == []:
        genre = [k for k in soup.find('meta', attrs={'name':'keywords'}).get('content').split(',') if k not in ['NHK','ニュース', 'NHK NEWS WEB']]
    keywords = json_data.get('keywords', [])
    article = soup.find('div', id="news_textbody").text
    if soup.find_all('div', id="news_textmore") != []:
        for textmore in soup.find_all('div', id="news_textmore"):
            article += ('\n' + textmore.text)
    if soup.find_all('div', class_="news_add") != []:
        for newsadd in soup.find_all('div', class_="news_add"):
            if newsadd.h3 != None:
                newsadd.h3.extract()
            article += ('\n' + newsadd.text)
            
    return {
        'id':url.split('/')[-1].split('.html')[0],
        'title':title,
        'article':article.strip(),
        'genre':genre,
        'keywords':keywords,
        'url':url,
        'datePublished':date,
        'dateModified':date_m
    }

# for old web normal
def make_date_normal_old(hmd,time):
    year, month, day = hmd[:4], hmd[4:6], hmd[6:]
    hour, minute = time.split('時')
    minute = minute.strip('分')
    if len(hour) == 1:
        hour = '0' + hour
    if len(minute) == 1:
        minute = '0' + minute
    return f"{year}-{month}-{day}T{hour}:{minute}"

def scrape_one_old(html, url):
    soup = BeautifulSoup(html, "html.parser")
    title = soup.find('span', class_="contentTitle").text.strip()
    hmd_ = url.split('/')[-2]
    time_ = soup.find('span', id="news_time").text
    date = make_date_normal_old(hmd_, time_)
    genre = [k for k in soup.find('meta', attrs={'name':'keywords'}).get('content').split(',') if k not in ['NHK','ニュース', 'NHK NEWS WEB','ＮＨＫ','ＮＨＫニュース','']]
    article = soup.find(['div','p'], id="news_textbody").text
    if soup.find_all(['div','p'], id="news_textmore") != []:
        for textmore in soup.find_all(['div','p'], id="news_textmore"):
            article += ('\n' + textmore.text)
    if soup.find_all(['div','p'], class_="news_add") != []:
        for newsadd in soup.find_all(['div','p'], class_="news_add"):
            if newsadd.h3 != None:
                newsadd.h3.extract()
            article += ('\n' + newsadd.text)
            
    return {
        'id':url.split('/')[-1].split('.html')[0],
        'title':title,
        'article':article.strip(),
        'genre':genre,
        'keywords':[],
        'url':url,
        'datePublished':date,
        'dateModified':""
    }

def get_archiveurl_from_id(ID, date):
    url1 = f'https://web.archive.org/web/*/https://www3.nhk.or.jp/news/html/{date}/k{ID}1000.html'
    url2 = f'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/html/{date}/k{ID}1000.html'
    
    driver.get(url1)
    time.sleep(3)
    html = str(driver.page_source.encode('utf-8'))
    snap = re.search(r'(times between|1 time|times).*?<a href="(.+?)">', html)
    archiveurl = 'https://web.archive.org' + snap.group(2)
    
    if 'nhk' not in archiveurl:
        driver.get(url2)
        time.sleep(3)
        html = str(driver.page_source.encode('utf-8'))
        snap = re.search(r'(times between|1 time|times).*?<a href="(.+?)">', html)
        archiveurl = 'https://web.archive.org' + snap.group(2)
    return None if 'nhk' not in archiveurl else archiveurl

def get_article_from_archiveurl(archiveurl):
    response = requests.get(archiveurl)
    time.sleep(2)
    html = response.text
    url_true = 'http' + archiveurl.split('/http')[-1]
    if 'This page is not available on the web' in html:
        return None
    try:
        try:
            dic = scrape_one_new(html, url_true)
        except:
            dic = scrape_one_old(html, url_true)
        return dic
    except:
        return None

def js(dic, year):
    if dic == None:
        return
    with open(f'nhkweb{year}.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open(f'nhkweb{year}.json', 'w', encoding='utf-8') as f:
        if dic['id'] not in [x['id'] for x in data]:
            data.append(dic)
        else:
            for i, d in enumerate(data):
                if dic['id'] == d['id']:
                    data[i] = dic
        data = sorted(data, key=lambda x:x['id'])
        json.dump(data, f, indent=4, ensure_ascii=False)

# check ID

In [10]:
data = pd.read_json('nhkweb2019.json')
data.head()

,id,title,article,genre,keywords,url,datePublished,dateModified
0,k10011760881000,僕、迷惑ですか？,東日本大震災のあと、「僕」は売れっ子になりました。売り上げは何倍にも跳ね上がり、カジュアルだ...,[暮らし],[],https://www3.nhk.or.jp/news/html/20190104/k100...,2019-01-04T12:32:01+09:00,2019-01-04T12:32:02+09:00
1,k10011764791000,日米欧もアジアも 2018年 株価下落 景気先行きに不安根強く,ニューヨーク株式市場は31日、2018年最後の取り引きが行われ、ダウ平均株価は値上がりして取...,[ビジネス],[株価・為替],http://www3.nhk.or.jp/news/html/20190101/k1001...,2019-01-01T08:55,
2,k10011764801000,ロシア スパイ容疑で米国人の男拘束 米ロ関係一層悪化へ,ロシアの治安当局は、スパイ活動をしていた疑いでアメリカ人の男をモスクワ市内で拘束しました。米...,[国際],[],https://www3.nhk.or.jp/news/html/20190101/k100...,2019-01-01T08:39,
3,k10011764811000,独 メルケル首相 米欧の自国第一主義に懸念 新年前に演説,ドイツのメルケル首相は新年を前にした恒例のテレビ演説で「国際協調という確かなものが圧力にさら...,[国際],[],http://www3.nhk.or.jp/news/html/20190101/k1001...,2019-01-01T09:02,
4,k10011764821000,マーライオンも摩天楼も 新年祝う花火が照らす,日本より１時間遅れで新年を迎えたシンガポールでは、観光名所として知られるマーライオンの像の前...,[国際],[国際５],http://www3.nhk.or.jp/news/html/20190101/k1001...,2019-01-01T09:13,


# get missing url

In [4]:
data = pd.read_json('nhkweb2019.json')
data = data[['id', 'url']]
data.id = data.id.apply(lambda x:x[1:-4])
data['date'] = data.url.apply(lambda x:x.split('news/html/')[-1].split('/')[0])
data[data.id == "1001176803"]

,id,url,date
297,1001176803,https://www3.nhk.or.jp/news/html/20190105/k100...,20190105


In [5]:
driver = webdriver.Chrome()

In [6]:
year = 2019

for i in range(297, 500):
    ID1, ID2 = data.iat[i,0], data.iat[i+1,0]
    date1, date2 = data.iat[i,2], data.iat[i+1,2]

    if int(ID1) + 1 == int(ID2): # continuous = no missing
        continue
    elif date1 == date2:  # not continuous, but in the same day
        print(ID1)
        for ID in range(int(ID1)+1, int(ID2)):
            archiveurl = get_archiveurl_from_id(ID, date1)
            if archiveurl != None:
                dic = get_article_from_archiveurl(archiveurl)
                js(dic, year)
    else:  # not continuous, not in the same day
        print(ID1)
        is_date1 = True
        for ID in range(int(ID1)+1, int(ID2)):
            if is_date1:
                archiveurl = get_archiveurl_from_id(ID, date1)
                if archiveurl:
                    dic = get_article_from_archiveurl(archiveurl)
                    js(dic, year)
                else:
                    archiveurl = get_archiveurl_from_id(ID, date2)
                    if archiveurl:
                        dic = get_article_from_archiveurl(archiveurl)
                        js(dic, year)
                        is_date1 = False
            else:
                archiveurl = get_archiveurl_from_id(ID, date2)
                if archiveurl:
                        js(dic, year)

1001176803
1001176842
1001176854
1001176865
1001176869
1001176871
1001176895
1001176919
1001176940
1001176946
1001176949
1001176952
1001176954
1001176960
1001176963
1001176969
1001176973
1001176997
1001177003
1001177024
1001177033
1001177039
1001177041
1001177043
1001177045
1001177049
1001177057
1001177076


In [ ]:
driver.page_source.encode("utf8")